# **Guide to Maximum Bipartite Matching in ViMMS (Updated 21/04/2024)**

This notebook is a guide to fragmentation strategies in ViMMS based on the use of maximum matching algorithms.

In [1]:
import sys
import os

user_vimms = os.path.join( # Put your path to your install of ViMMS here!
    "C:\\",
    "Users",
    "mcbrider5002",
    "Desktop",
    "Workspace",
    "phd",
    "peak_picking",
    "vimms"
)

sys.path.append(user_vimms)

### **What is "Maximum Bipartite Matching"?**

Computing scientists study many types of allocation problems in order to develop algorithms which compute an optimal assignment of some resource, and perform this computation with an efficient use of computing power. One such class of problem is a "matching" problem where you have a list of parties, or resources, and a list of preference or compatibility criteria, and you must make pairs of them such that no individual is double-allocated. A maximum matching is a list of these pairs such that no list with more pairs exists for a given input problem. 

This can be represented pictorially like so:

<img src="images/bipartite_matching.svg" width=360 height=360 style="margin:auto"/>

This formalism is a computational graph; the circles or "vertices" represent an individual instance of our abstract "resource"; and the lines joining them, or "edges", represent compatibility between the two, meaning we can form a pair between them. This graph has a special structure where there are two groups of vertices and all edges join a vertex in one group to another: this structure is a bipartite graph. A maximum bipartite matching is just a matching computed on a graph with this structure: the extra restrictions mean a prospective algorithm has to consider less possible cases and thus we can deploy a more efficient algorithm to this problem compared to the general matching problem.

The lines marked in red highlight a particular candidate matching. Each edge chosen for the matching must, by definition, not touch a vertex touched by any other edge included in the matching: we can see this requirement is satisfied. Additionally, all scans from one side of the graph are assigned so this matching is "full" or "one-sided perfect". It follows trivially that it must also be a maximum matching: all vertices from one side of the graph have been assigned, and the other group of vertices has only edges that lead to these already-assigned vertices.

### **Why Matching?**

So how does this relate to fragmentation strategies? The relationships between MS2 scans and target chromatographic peaks can be modelled as such a bipartite graph. Each MS2 scan and each chromatographic peak becomes a vertex on opposite sides of the bipartite graph. Then, an edge exists between a given pair of vertices if one is an MS2 scan and the other is a chromatographic peak for which that MS2 scan would be able to target to acquire a fragmentation spectrum. Having defined this construct we can efficiently resolve the assignments of MS2s to peaks such that the maximum number of peaks possible are targeted.

It would, of course, be easy to perform the above example by hand. But in LC-MS/MS data you might have thousands or tens of thousands of MS2 scans and peaks, and millions of edges joining the two. Even for a computer program with a well-structured problem definition like this, this task is not trivial and may be time-consuming. By leveraging efficient algorithms designed for the maximum bipartite matching problem, like the Hopcroft-Karp algorithm, we can use someone else's implementation not written for this specific problem to solve hard instances like these. We need only construct the appropriate graph structure first.

To construct this graph in ViMMS you need three pieces of input data: a _scan schedule_, a _target list_ and some _representative data_. The _scan schedule_ specifies what order to run MS1 and MS2 scans in, and their expected durations: the MS2 scans here will form one set of vertices. The _target list_ is a list of bounding boxes in (RT, m/z) space, one per peak you would like to target: these form the other set of vertices. The _representative data_ is some previous observation of the data (preferably one MS1-only fullscan .mzML per sample you would like to run) from which we can see intensity readings for each peak-box at a given point in time. This then allows us to construct edges: an MS2-peak pair has an edge if the MS2 overlaps with the peak-box in time _and_ the MS2 has a preceding MS1 with an intensity reading above some minimum threshold inside the peak-box.

Having constructed our graph, we can call our standard algorithm on it to get a maximum matching. That maximum matching can be utilised in several ways for fragmentation strategies. For example, we can directly run a "pre-scheduled" method by running the exact schedule we gave the matching algorithm and targeting MS2 scans at the peak they were paired with in the matching. Or, we can create inclusion windows for a DDA method by taking the scan time of any MS2 scan included in the matching and drawing a fixed-size window around it. An overview of the whole process can be seen below:

<img src="images/matching_workflow.svg" width=820 height=820 style="margin:auto"/>

In our work we have also extended this basic bipartite matching concept to multiple samples (by creating one graph per run and combining them), intensity acquisition optimisation (by marking each edge with the intensity value of that acquisition, and solving the maximum weighted bipartite matching to find the matching with the greatest sum of these values) and to full assignments (by using heuristic rules to assign any leftover scans not included in the matching). Using these techniques is also covered in this notebook.

Do note that there are other kinds of matching problem: for example, the Stable Marriage problem, rather than just indicating a binary compatibility between two vertices, has each vertex come with a preference list stating its order of preference 

## **Quick-Start**

The easiest way to use a `Matching`-based fragmentation strategy is the `make_matching` convenience method. Let's run an `Experiment` with TopN Exclusion, a pre-scheduled matching and TopNEx with DDA inclusion windows.

In [2]:
from vimms.Common import POSITIVE
from vimms.Roi import RoiBuilderParams
from vimms.PeakPicking import XCMSScriptParams
from vimms.Matching import MatchingScan, Matching
from vimms.BoxManager import BoxManager
from vimms.Experiment import ExperimentCase, Experiment
from vimms.Controller.misc import TaskFilter

 C:\Users\mcbrider5002\anaconda3\lib\site-packages\psims\mzmlb\writer.py:33: UserWarning:hdf5plugin is missing! Only the slower GZIP compression scheme will be available! Please install hdf5plugin to be able to use Blosc.


In [3]:
# Define some stuff for the experiment in general and the DDA controllers
# See the Guide to ViMMS for more information on basic usage

out_dir = os.path.join("results", "matching")
xcms_r_script = os.path.join(user_vimms, "vimms", "scripts", "xcms_script.R")

num_workers = 8
ionisation_mode = POSITIVE
min_rt, max_rt = 0, 1440
isolation_width = 1
intensity_threshold = 5000
scan_duration_dict = {1: 0.59, 2: 0.19}

topN_params = {
    "ionisation_mode" : ionisation_mode,
    "N" : 20,
    "isolation_width" : isolation_width,
    "min_ms1_intensity" : intensity_threshold,
    "mz_tol" : 10,
    "rt_tol" : 60
}

topNEXt_params = {
    **topN_params,
    "min_roi_length_for_fragmentation" : 0,
    "roi_params" : RoiBuilderParams(
        min_roi_intensity=0,
        min_roi_length=3,
    )
}

pp_params = XCMSScriptParams(
    xcms_r_script = xcms_r_script,
)

beer_fullscans = [os.path.join("fixtures", f"fullscan_beer{i}_0.mzML") for i in range(1, 3)]

In [4]:
# Make a Matching object, which can be used directly for a pre-scheduled controller

# Representative data, i.e. the run structure we would like to have...
fullscan_paths = beer_fullscans * 2

# What times we expect to run scans at, and at what MS level
times_list = [list(MatchingScan.topN_times(N, max_rt, scan_duration_dict)) for N in [20] * len(fullscan_paths)]

# Use a peak-picker on the representative data to define our target list
aligned_file = pp_params.pick_aligned_peaks(
    input_files = fullscan_paths,
    output_dir = out_dir,
    output_name = f"{len(fullscan_paths)}_beer_peak_picked.csv",
    force = False
)

shared_matching_params = {
    "fullscan_paths" : fullscan_paths, # Representative data per each LC-MS/MS run to do
    "times_list" : times_list, # Times and levels for scans to be run
    "aligned_reader" : XCMSScriptParams, # How to read the target list format
    "aligned_file" : aligned_file, # Peak-picked file
    "ionisation_mode" : ionisation_mode,
    "intensity_threshold" : intensity_threshold # Only create edges above this threshold
}

matching = Matching.make_matching( # Create a Matching object with desired properties
    **shared_matching_params,
    weighted=Matching.TWOSTEP, # Optimise coverage then acquisition intensities
    full_assignment_strategy=Matching.RECURSIVE_ASSIGNMENT # Assign leftover scans by repeatedly running matchings on them
)

matching.log.summarise() # Show some interesting information about the Matching

16942 aligned boxes contained in file


2024-04-24 12:26:42.961 | DEBUG    | vimms.Chemicals:_extract_rois:834 - Extracted 291410 good ROIs from fixtures\fullscan_beer1_0.mzML
2024-04-24 12:30:12.327 | DEBUG    | vimms.Chemicals:_extract_rois:834 - Extracted 318956 good ROIs from fixtures\fullscan_beer2_0.mzML


matching_size: 12422
chem_count: 16942
scan_count: 26240
edge_count: 4348654
chems_above_threshold: 15681
start_scan: 2024-04-24 12:23:45.274803
end_scan: 2024-04-24 12:31:29.597922
start_chem: 2024-04-24 12:31:30.706579
end_chem: 2024-04-24 12:31:30.880215
start_matching: 2024-04-24 12:31:30.888256
end_matching: 2024-04-24 12:34:50.307590
start_assign: 2024-04-24 12:34:50.307590
end_assign: 2024-04-24 12:36:45.275396


In [5]:
# Turn the matching object into inclusion windows of fixed size and slot it into a TopNEXt BoxManager

inclusion_boxes = matching.make_inclusion_boxes(rt_width=10, mz_width=10)
grid_base = BoxManager(inclusion_boxes=inclusion_boxes)

In [6]:
# Run the Experiment

tfilter = TaskFilter(scan_duration_dict[1], scan_duration_dict[2]) # Add TaskFilter for dynamic rescheduling


exp = Experiment()
exp.add_cases([
    ExperimentCase("topN_exclusion", fullscan_paths, topN_params),
    ExperimentCase("matching", fullscan_paths,  {"isolation_width": isolation_width, "task_filter": tfilter}, shareable_base=matching, name="two_step_matching_with_recursive_assignment"),
    ExperimentCase("topNEX", fullscan_paths, topNEXt_params, shareable_base=grid_base, name="topNEX_inclusion"),
])

exp.run_experiment(
    out_dir=out_dir,
    min_rt=min_rt,
    max_rt=max_rt,
    ionisation_mode=ionisation_mode,
    scan_duration_dict=scan_duration_dict,
    overwrite_keyfile=False,
    point_noise_threshold=0,
    chem_noise_threshold=0,
    num_workers=num_workers
)

Creating Chemicals...

Running Experiment of 3 cases...


In [7]:
# Evaluate the Experiment and show some results

exp.evaluate(
    pp_params=pp_params,
    num_workers=num_workers,
    isolation_widths=isolation_width,
    aligned_names=f"{len(fullscan_paths)}_beer_peak_picked.csv",
    force_peak_picking = False,
    check_files = "exact"
)

exp.summarise(
    num_workers=num_workers,
    min_intensities=intensity_threshold,
    rank_key = "cumulative_intensity_proportion"
)

16942 aligned boxes contained in file
16942 aligned boxes contained in file
16942 aligned boxes contained in file

two_step_matching_with_recursive_assignment
Number of chems above min intensity: 15681
Number of fragmentations: [6560, 6560, 6560, 6560]
Cumulative coverage: [4703, 9237, 11426, 13603]
Cumulative coverage proportion: [0.2999170971239079, 0.5890568203558446, 0.7286525094062879, 0.867482941138958]
Cumulative intensity proportion: [0.21935683419503332, 0.44876641379529536, 0.5638165494658173, 0.6881781131630043]
Cumulative intensity proportion of covered spectra: [0.7313915621969631, 0.7618389233218607, 0.7737797402567373, 0.7933044911055701]
Times covered: {0: 3339, 1: 8325, 2: 4384, 3: 565, 4: 329}
Times fragmented: {0: 2737, 1: 6005, 2: 3520, 3: 1692, 4: 740, 5: 493, 6: 401, 7: 320, 8: 207, 9: 124, 10: 85, 11: 64, 12: 60, 13: 44, 14: 65, 15: 56, 16: 43, 17: 30, 18: 29, 19: 26, 20: 18, 21: 25, 22: 17, 23: 5, 24: 12, 25: 7, 26: 3, 27: 4, 28: 4, 29: 6, 30: 8, 31: 3, 32: 13, 

# **Matching In-Depth**

The `make_matching` function automates a lot of the steps but it is possible to manually initialise the `Matching` instead. This involves creation of both sets of vertices as `MatchingScans` and `MatchingChems` and then initialisation of the matching itself through `multi_schedule2graph`.

First, though, let's import some things, define our input data and define a logging object so we can pass it to constructors (we'll see how to use it later).

In [8]:
from vimms.Common import POSITIVE
from vimms.PeakPicking import XCMSScriptParams
from vimms.Matching import MatchingScan, MatchingChem, Matching, MatchingLog

out_dir = os.path.join("results", "matching")
xcms_r_script = os.path.join(user_vimms, "vimms", "scripts", "xcms_script.R")

ionisation_mode = POSITIVE
min_rt, max_rt = 0, 1440
isolation_width = 1
intensity_threshold = 5000
scan_duration_dict = {1: 0.59, 2: 0.19}

beer_fullscans = [os.path.join("fixtures", f"fullscan_beer{i}_0.mzML") for i in range(1, 3)]
fullscan_paths = beer_fullscans * 2

log = MatchingLog()

#### **MatchingScans**

A MatchingScan represents what an individual scan is expected to look like in an upcoming scan schedule. An MS1 scan has an RT and a list of (m/z, intensity) pairs which are created by interpolating scans in the representative data. MS2 scans only have the RT and will become vertices in the graph. We need one scan schedule per each LC-MS/MS run we plan to do.

We already defined the representative data, so now we just need to define the scan schedule. We can make one straightforwardly with the `topN_times` method, but for the sake of demonstration let's do it manually.

In [9]:
N = 20
scan_idx = 1
times_list = [(1, 0.0)]
last_level, last_time = times_list[-1]

while(last_time < max_rt):
    new_level = 1 if scan_idx % N == 0 else 2
    new_rt = last_time + scan_duration_dict[last_level]
    times_list.append((new_level, new_rt))
    
    last_level, last_time = times_list[-1]
    scan_idx += 1

times_list = [times_list] * len(fullscan_paths)

scans_list = MatchingScan.mzmls2scans(
    fullscan_paths,
    times_list, 
    ionisation_mode, 
    log=log
)

2024-04-24 12:58:37.918 | DEBUG    | vimms.Chemicals:_extract_rois:834 - Extracted 291410 good ROIs from fixtures\fullscan_beer1_0.mzML
2024-04-24 13:02:09.206 | DEBUG    | vimms.Chemicals:_extract_rois:834 - Extracted 318956 good ROIs from fixtures\fullscan_beer2_0.mzML


### **MatchingChems**

`MatchingChems` represent the targets for the matching method to aim MS2 scans at. They're specified as a rectangle in (RT, m/z) space, and form the other side of the bipartite graph from the MS2 `MatchingScans`. The target list should specify where each target is expected to be per each run (their absence in a given run is allowed) so that we define an $m\times{}n$ matrix where the row index gives the specific target and the column index gives the run, or vice-versa. 

We'll pick peaks, align them, and then read the peak-list back in from the output file, but any method of generating targets suffices given a file listing them and an object which knows how to read the file format.

In [10]:
pp_params = XCMSScriptParams(
    xcms_r_script = xcms_r_script,
)

aligned_file = pp_params.pick_aligned_peaks(
    input_files = fullscan_paths,
    output_dir = out_dir,
    output_name = f"{len(fullscan_paths)}_beer_peak_picked.csv",
    force = False
)

chems_list = MatchingChem.boxfile2nodes(
    XCMSScriptParams, # File reader
    aligned_file,
    fullscan_paths,
    log=log
)

16942 aligned boxes contained in file


### **Matching**

Now we have both the `MatchingScans` and `MatchingChems` (one list of each per LC-MS/MS run we want to perform) we can construct our bipartite graph and solve a matching. First, edges are constructed. For each run we overlay the `MatchingScans` with the `MatchingChems`. Then for each MS2 `MatchingScan` we create an edge between it and any `MatchingChem` if all of the following conditions are met:

    * The MS2 `MatchingScan` intersects the `MatchingChem`'s box on the RT dimension;
    * The MS2 `MatchingScan` has a precursor MS1 `MatchingScan` with a point also within the (RT, m/z) bounds of the `MatchingChem`'s box;
    * That precursor point is above a minimum intensity threshold.

That edge is then labelled with the intensity of the precursor point, to allow optimising acquisition intensity. We now have all the objects we would need to instantiate one bipartite graph per injection, but to plan for the whole acquisition they must be combined first. `MatchingScans` are simply transferred to the new graph as-is (while tracking the index of the run they came from) but `MatchingChems` are combined using the alignment information written into the data file they were read from. All edges continue to persist between a `MatchingScan` and an aligned `MatchingChem` where the edge would join that `MatchingScan` and one of the `MatchingChems` combined into the aligned `MatchingChem` by this process. After this, we pass these objects to NetworkX and let it solve the maximum bipartite matching for us. If we also wish to optimise acquisition intensity, then an auxiliary graph is created using only the `MatchingChems` included in that maximum matching, and a maximum weighted bipartite matching is solved on that graph. (Deleting excess `MatchingChems` is necessary due to an implementation detail of NetworkX.)

All of these steps can be performed internally by `multi_schedule2graph`!

In [11]:
intensity_threshold = 5000

basic_matching = Matching.multi_schedule2graph(
    scans_list,
    chems_list,
    intensity_threshold,
    edge_limit=None, # If you need this to run faster, you can set this to the maximum number of edges allowed on any vertex, and clip any extras, starting from the lowest intensity
    weighted=Matching.TWOSTEP,
    full_assignment_strategy=Matching.MATCHING_ONLY,
    log=log
)

It's also possible to use the parameters to control whether the matching is weighted (i.e. whether we optimise acquisition intensity) and how we assign any leftover scans that weren't included in the matching. `weighted` controls the mode for weighted assignment, and `full_assignment_strategy` controls how the leftover scans are made into a full assignment.

It's also possible to change the full assignment after the `Matching` object is created via `assign_remaining_scans`. Like so:

In [12]:
import copy

recursive_matching = copy.deepcopy(basic_matching) # We're going to show the difference shortly so we don't want to modify the original object
recursive_matching.assign_remaining_scans(Matching.RECURSIVE_ASSIGNMENT)

These objects can also be freely turned into inclusion windows as follows:

In [13]:
from vimms.BoxManager import BoxManager

inclusion_boxes = basic_matching.make_inclusion_boxes(rt_width=10, mz_width=10)
grid_base = BoxManager(inclusion_boxes=inclusion_boxes) # Needed to use them with a TopNEXt controller, but you could do something else with them instead...

Note that only items in the matching itself are used for inclusion windows, not the whole assignment. 

Also, the `Matching` continues to store intermediate steps like the constructed NetworkX graph internally. While this can be useful, sometimes these data can be quite large - by calling the `strip` method you can delete them and free memory space. Do note that this will prevent operations that require these to still be present, like recomputing the full assignment.

### **MatchingLog**

The `MatchingLog` object we've been passing around can give us some useful information about the matching we computed. First let's call `matching_report` to fill out some of the information it tracks, then let's see what it can show us with `summarise`.

In [14]:
basic_matching.matching_report()
log.summarise()

matching_size: 12387
chem_count: 16942
scan_count: 26060
edge_count: 4331608
chems_above_threshold: 15702
start_scan: 2024-04-24 12:55:29.404268
end_scan: 2024-04-24 13:03:23.881703
start_chem: 2024-04-24 13:03:25.064226
end_chem: 2024-04-24 13:03:26.588461
start_matching: 2024-04-24 13:03:26.621160
end_matching: 2024-04-24 13:06:36.333119
start_assign: 2024-04-24 13:06:36.333119
end_assign: 2024-04-24 13:06:36.353476


Of course, the recursive matching we deepcopied has its own log:

In [15]:
recursive_matching.matching_report()
recursive_matching.log.summarise()

matching_size: 12387
chem_count: 16942
scan_count: 26060
edge_count: 4331608
chems_above_threshold: 15702
start_scan: 2024-04-24 12:55:29.404268
end_scan: 2024-04-24 13:03:23.881703
start_chem: 2024-04-24 13:03:25.064226
end_chem: 2024-04-24 13:03:26.588461
start_matching: 2024-04-24 13:03:26.621160
end_matching: 2024-04-24 13:06:36.333119
start_assign: 2024-04-24 13:07:28.364436
end_assign: 2024-04-24 13:09:26.878849


We can also specify which fields we want to get out of summarise - not all are enabled by default. e.g.

In [16]:
recursive_matching.log.summarise(["matching_size", "recursive_scan_counts"])

matching_size: 12387
recursive_scan_counts: [25976, 13589, 10686, 8655, 7012, 5518, 4369, 3535, 3031, 2733, 2465, 2234, 2012, 1803, 1620, 1445, 1324, 1236, 1150, 1069, 989, 910, 832, 757, 694, 634, 578, 523, 473, 425, 380, 336, 294, 259, 232, 208, 184, 160, 136, 112, 90, 68, 46, 24, 6]


### **MatchingController and TaskFilter**

The pre-scheduled matching is weak to unexpected changes in its plan, and can even become completely desynchronised. However, the `MatchingController` (like other pre-scheduled controllers in ViMMS) can use a task filtering object to dynamically adjust its schedule (likely in simpler ways than computing the original plan). Experiment therefore also allows passing a `task_filter` to matching controllers.

Let's define one using the base `TaskFilter` class and test it in an experiment where scan times are consistently longer than expected...

In [17]:
from vimms.Controller.misc import TaskFilter

tfilter = TaskFilter(
    scan_duration_dict[1], # NB this is still conditioned on the expected times for scans
    scan_duration_dict[2],
    skip_margin=0.5, # Default value, how aggressively we skip missed items
    add_margin=1.2 # Default value, how aggressively we pad schedule with items to pass time 
)

true_duration_dict = {level: 1.1 * t for level, t in scan_duration_dict.items()}

In [18]:
# Define DDA stuff again...

from vimms.Roi import RoiBuilderParams

topN_params = {
    "ionisation_mode" : ionisation_mode,
    "N" : 20,
    "isolation_width" : isolation_width,
    "min_ms1_intensity" : intensity_threshold,
    "mz_tol" : 10,
    "rt_tol" : 60
}

topNEXt_params = {
    **topN_params,
    "min_roi_length_for_fragmentation" : 0,
    "roi_params" : RoiBuilderParams(
        min_roi_intensity=0,
        min_roi_length=3,
    )
}

In [19]:
# Run the new Experiment

from vimms.Experiment import ExperimentCase, Experiment

num_workers = 8

exp = Experiment()
exp.add_cases([
    ExperimentCase("matching", fullscan_paths,  {"isolation_width": isolation_width}, shareable_base=basic_matching, name="two_step_matching"),
    ExperimentCase("matching", fullscan_paths,  {"isolation_width": isolation_width, "task_filter": tfilter}, shareable_base=basic_matching, name="adaptive_two_step_matching"),
    ExperimentCase("matching", fullscan_paths,  {"isolation_width": isolation_width}, shareable_base=recursive_matching, name="two_step_matching_with_recursive_assignment"),
    ExperimentCase("matching", fullscan_paths,  {"isolation_width": isolation_width, "task_filter": tfilter}, shareable_base=recursive_matching, name="adaptive_two_step_matching_with_recursive_assignment"),
    ExperimentCase("topNEX", fullscan_paths, topNEXt_params, shareable_base=grid_base, name="topNEX_inclusion"),
])

exp.run_experiment(
    out_dir=out_dir,
    min_rt=min_rt,
    max_rt=max_rt,
    ionisation_mode=ionisation_mode,
    scan_duration_dict=true_duration_dict,
    overwrite_keyfile=False,
    point_noise_threshold=0,
    chem_noise_threshold=0,
    num_workers=num_workers
)

Creating Chemicals...

Running Experiment of 5 cases...


In [20]:
# Evaluate the Experiment and show some results

exp.evaluate(
    pp_params=pp_params,
    num_workers=num_workers,
    isolation_widths=None,
    aligned_names=f"{len(fullscan_paths)}_beer_peak_picked.csv",
    force_peak_picking = False,
    check_files = "exact"
)

exp.summarise(
    num_workers=num_workers,
    min_intensities=intensity_threshold,
    rank_key = "cumulative_intensity_proportion"
)

16942 aligned boxes contained in file
16942 aligned boxes contained in file
16942 aligned boxes contained in file
16942 aligned boxes contained in file
16942 aligned boxes contained in file

adaptive_two_step_matching_with_recursive_assignment
Number of chems above min intensity: 15717
Number of fragmentations: [5825, 5825, 5825, 5825]
Cumulative coverage: [3422, 6681, 8836, 11143]
Cumulative coverage proportion: [0.2177260291404212, 0.42508112235159384, 0.562193802888592, 0.7089775402430489]
Cumulative intensity proportion: [0.1687237338429701, 0.33403116854739234, 0.4371519308412744, 0.5486226887142749]
Cumulative intensity proportion of covered spectra: [0.774935980365272, 0.78580569915572, 0.7775822653952366, 0.7738223816317201]
Times covered: {0: 5799, 1: 8907, 2: 1871, 3: 193, 4: 172}
Times fragmented: {0: 5609, 1: 8637, 2: 789, 3: 357, 4: 246, 5: 382, 6: 298, 7: 250, 8: 114, 9: 25, 10: 19, 11: 16, 12: 29, 13: 26, 14: 38, 15: 20, 16: 6, 17: 2, 18: 1, 19: 3, 20: 8, 21: 6, 22: 5, 2

Firstly, we can see that recomputing the full assignment (on the same original object) improved the result in this unexpected environment. Secondly, we can see that a persistent lengthening of scan times causes systemic failure for the pre-scheduled matching, but this can addressed with `TaskFilter`!